In [38]:
import pyodbc
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [39]:
#  Conexión a **Azure SQL**
AZURE_SERVER = 'uaxmathfis.database.windows.net'
AZURE_DATABASE = 'usecases'
AZURE_DRIVER = '{ODBC Driver 17 for SQL Server}'

azure_conn_str = f"DRIVER={AZURE_DRIVER};SERVER={AZURE_SERVER};DATABASE={AZURE_DATABASE};Authentication=ActiveDirectoryInteractive"

In [40]:
# Parámetros de conexión
LOCAL_SERVER = 'localhost'
LOCAL_DATABASE = 'dwh_case1'
LOCAL_DRIVER = '{ODBC Driver 17 for SQL Server}'

# Conexión a SQL Server local
local_conn_local = f"DRIVER={LOCAL_DRIVER};SERVER={LOCAL_SERVER};DATABASE={LOCAL_DATABASE};Trusted_Connection=yes"

In [41]:
query_folder = "../database/sql"
# Lista de archivos SQL y nombre de tabla destino
tablas = {
    "dim_clientes": "dim_clientes.sql",
    "dim_tiempo": "dim_tiempo.sql",
    "dim_producto": "dim_producto.sql",
    "dim_zona": "dim_zona.sql",
    "tabla_hechos": "tabla_hechos.sql"
}

In [42]:
# Primary keys for each table.
primary_keys = {
    "tabla_hechos": ["CODE"],
    "dim_clientes": ["Customer_ID"],
    "dim_zona": ["TIENDA_ID"],
    "dim_producto": ["Id_Producto"],
    "dim_tiempo": ["Fecha"]
}

In [43]:
# Foreign keys for each table.
foreign_keys = {
    "tabla_hechos": {
        "Customer_ID": "dim_clientes(Customer_ID)",
        "TIENDA_ID": "dim_zona(TIENDA_ID)",
        "Id_Producto": "dim_producto(Id_Producto)",
        "Sales_Date": "dim_tiempo(Fecha)"
    }
}

In [44]:
def create_table_sql(table_name, df):
    # Definición de los tipos de datos SQL para cada columna del DataFrame: Por defecto tipo TEXTO.
    col_defs = []
    for col in df.columns:
        if np.issubdtype(df[col].dtype, np.datetime64):
            col_defs.append(f'[{col}] DATE')
        elif df[col].dtype == np.float32:
            col_defs.append(f'[{col}] FLOAT')
        elif df[col].dtype == np.int32:
            col_defs.append(f'[{col}] INT')
        else:
            col_defs.append(f'[{col}] NVARCHAR(255)')

    # Agregación clave primaria si corresponde.
    pk = ", PRIMARY KEY (" + ", ".join(primary_keys[table_name]) + ")" if table_name in primary_keys else ""
    # Agregación claves foráneas si corresponde.
    fk = ""
    if table_name in foreign_keys:
        for col, ref in foreign_keys[table_name].items():
            fk += f", FOREIGN KEY ({col}) REFERENCES {ref}"

    return f"CREATE TABLE {table_name} ({', '.join(col_defs)}{pk}{fk});"

In [45]:
def drop_tables_in_order(cursor, conn):
    drop_order = ["tabla_hechos", "dim_tiempo", "dim_producto", "dim_zona", "dim_clientes"]
    for table in drop_order:
        # Verifica si la tabla existe en el esquema actual.
        check_exists_query = f"""
        IF OBJECT_ID('{table}', 'U') IS NOT NULL
            DROP TABLE {table};
        """
        try:
            cursor.execute(check_exists_query)
            conn.commit()
        except Exception as e:
            print(f"Error al intentar eliminar la tabla {table}: {e}")

In [46]:
try:
    # Conexión a las bases de datos.
    conn_azure = pyodbc.connect(azure_conn_str)
    conn_local = pyodbc.connect(local_conn_local)
    print("Conexiones correctamente establecidas.\n")

    with conn_local.cursor() as cursor:
        drop_tables_in_order(cursor, conn_local)
    # Procesamiento de cada tabla definida en el diccionario de Queries.
    for table_name, file in tablas.items():
        print(f"Procesando: {table_name}")
        query_path = os.path.join(query_folder, file)
        with open(query_path, "r", encoding="utf-8") as f:
            sql_query = f.read()

        # Ejecución de la consulta sobre la base de datos de Azure.
        df = pd.read_sql(sql_query, conn_azure)

        # Eliminación de las columnas duplicadas.
        if df.columns.duplicated().any():
            print(f"Columnas duplicadas en {table_name}: {df.columns[df.columns.duplicated()].tolist()}")
            df = df.loc[:, ~df.columns.duplicated()]

        # Detección de las columnas tipo DATE para convertirlas adecuadamente.
        for col in df.columns:
            if df[col].dtype == object or df[col].dtype == "string":
                sample_values = df[col].astype(str).sample(min(len(df), 30), random_state=42)
                # Saltar si parece una columna numérica (para no confundir INT con DATE).
                if sample_values.str.isdigit().mean() > 0.8:
                    continue
                try:
                    parsed = pd.to_datetime(sample_values, errors='coerce')
                    if parsed.notna().sum() > 0.9 * len(sample_values):
                        df[col] = pd.to_datetime(df[col], errors='coerce')
                except:
                    pass
        # Si el DataFrame está vacío, se salta.
        if df.empty:
            print(f"La tabla {table_name} no devolvió resultados.\n")
            continue
        print(f"   - Filas obtenidas: {df.shape[0]}")
        print(f"   - Columnas: {df.columns.tolist()}")

        # Limpieza de valores nulos y tipos de datos.
        for col in df.columns:
            df[col] = df[col].replace(r'^\s*$', np.nan, regex=True) # Reemplazar espacios en blanco por NaN.
            if pd.api.types.is_numeric_dtype(df[col]):
                # Valor sentinel (ej: -1 o 999999).
                sentinel = -1
                df[col] = df[col].fillna(sentinel)
            elif pd.api.types.is_datetime64_any_dtype(df[col]):
                df[col] = df[col].fillna(df[col].mode(dropna=True)[0])
            else:
                df[col] = df[col].fillna("N/A")
        for col in df.select_dtypes(include=['float64']).columns:
            df[col] = df[col].astype(np.float32)
        for col in df.select_dtypes(include=['int64']).columns:
            df[col] = df[col].astype(np.int32)

        # Creación de la tabla en la base de datos local.
        with conn_local.cursor() as cursor:
            create_sql = create_table_sql(table_name, df)
            cursor.execute(create_sql)
            conn_local.commit()
            print(f"   - Tabla {table_name} creada correctamente.")

            placeholders = ', '.join(['?' for _ in df.columns])
            insert_sql = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.fast_executemany = True
            cursor.executemany(insert_sql, df.values.tolist())
            conn_local.commit()
            print(f"   - {df.shape[0]} filas insertadas.\n")

except Exception as e:
    print(f"Error: {e}")

finally:
    if 'conn_azure' in locals():
        conn_azure.close()
    if 'conn_local' in locals():
        conn_local.close()

print("ETL completado.")

Conexiones correctamente establecidas.

Procesando: dim_clientes
   - Filas obtenidas: 44053
   - Columnas: ['Customer_ID', 'EDAD', 'GENERO', 'RENTA_MEDIA_ESTIMADA', 'STATUS_SOCIAL', 'CODIGO_POSTAL', 'poblacion', 'provincia', 'Fecha_nacimiento', 'CP_NUM', 'Mosaic_number', 'Max_Mosaic']
   - Tabla dim_clientes creada correctamente.
   - 44053 filas insertadas.

Procesando: dim_tiempo
   - Filas obtenidas: 3652
   - Columnas: ['Fecha', 'Año', 'Añomes', 'Mes', 'Dia', 'Diadelasemana', 'Diadelesemana_desc', 'Festivo', 'Findesemana', 'FinMes', 'InicioMes', 'Laboral', 'Mes_desc', 'Semana']
   - Tabla dim_tiempo creada correctamente.
   - 3652 filas insertadas.

Procesando: dim_producto
   - Filas obtenidas: 404
   - Columnas: ['Id_Producto', 'Code_', 'Modelo', 'Kw', 'TIPO_CARROCERIA', 'TRANSMISION_ID', 'FUEL', 'CATEGORIA_ID', 'Equipamiento', 'Margen', 'Costetransporte', 'Margendistribuidor', 'GastosMarketing']
   - Tabla dim_producto creada correctamente.
   - 404 filas insertadas.

Procesand